In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
import polars as pl
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse._matrix import spmatrix
from amazon.models import *
import polars as pl
import numpy as np
import joblib
from plotly.graph_objs._figure import Figure
import nltk
from nltk.corpus import stopwords
import re
import gc


# 1. Download & grab the NLTK stop‐word list once
# nltk.download("stopwords")
# STOP_WORDS = set(stopwords.words("english"))
# STOP_WORDS.add("amazon")
# STOP_WORDS.add("Amazon")
# STOP_WORDS.add("review")

In [3]:
# stop_pattern = r"\b(" + "|".join(map(re.escape, STOP_WORDS)) + r")\b"

# lf: pl.LazyFrame = (
#     pl.scan_parquet("data/processed/amazon-2023.parquet")
#       .select([
#           pl.col("rating").cast(pl.Int8),
#           pl.col("text").cast(pl.Utf8)
#       ])
#       .with_columns((pl.col("rating") > 3).cast(pl.Int8).alias("rating"))
#       .with_columns(
#           pl.col("text")
#             .str.replace_all(stop_pattern, "", literal=False)
#             .str.replace_all(r"\s+", " ", literal=False)  # collapse any gaps left
#             .str.strip_chars()
#             .alias("text")
#       )
#       .with_columns(
#           pl.col("text")
#             .str.replace_all(r"http\S+|www\.\S+", "", literal=False)
#             .str.replace_all(r"[^\w\s]", "", literal=False)
#             .str.replace_all(r"\s+", " ", literal=False)
#             .str.to_lowercase()
#             .str.strip_chars()
#             .str.slice(0, 128)
#             .alias("text")
#       )
#       .filter(pl.col("text").str.len_chars() > 10)
# )

# lf.sink_parquet("data/processed/training.parquet", engine="streaming")

In [4]:
# df=pl.read_parquet("data/processed/training.parquet", low_memory=True, memory_map=True, use_pyarrow=True)

In [5]:
# X:pl.Series = df["text"]
# y: pl.Series = df["rating"]

# del df
# gc.collect()

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, shuffle=True
# )

In [7]:
# pl.DataFrame({"text": X_train}).write_parquet("data/processed/X_train.parquet")
# del X_train
# gc.collect()

In [8]:
# pl.DataFrame({"rating": y_train}).write_parquet("data/processed/y_train.parquet")
# del y_train
# gc.collect()

In [9]:
# pl.DataFrame({"text": X_test}).write_parquet("data/processed/X_test.parquet")
# del X_test
# gc.collect()

In [10]:
# pl.DataFrame({"rating": y_test}).write_parquet("data/processed/y_test.parquet")
# del y_test
# gc.collect()

In [ ]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    token_pattern=r'\b\w+\b', 
    min_df=5,
    max_df=0.8,
    stop_words="english"
)


X_train = pl.read_parquet("data/processed/X_train.parquet",low_memory=True, memory_map=True, use_pyarrow=True)
X_train: spmatrix = vectorizer.fit_transform(X_train["text"])

In [ ]:
y_train = pl.read_parquet("data/processed/y_train.parquet",low_memory=True, memory_map=True, use_pyarrow=True)
y_train = y_train["rating"]

model = LogisticRegression(max_iter=100) # iter added to stop warning
model.fit(X_train, y_train)

In [ ]:
del X_train
del y_train
gc.collect()
joblib.dump(model, "data/logreg_tfidf.pkl")

In [ ]:
# vectorizer = TfidfVectorizer(
#     lowercase=True,
#     token_pattern=r'\b\w+\b', 
#     min_df=5,
#     max_df=0.8,
#     stop_words="english"
# )

# X_test = pl.read_parquet("data/processed/X_test.parquet",low_memory=True, memory_map=True, use_pyarrow=True)["text"]
# X_test: spmatrix = vectorizer.transform(X_test)

In [ ]:
# y_pred: np.ndarray = model.predict(X_test)
# y_test = pl.read_parquet("data/processed/y_test.parquet",low_memory=True, memory_map=True, use_pyarrow=True)["rating"]


# metrics: dict = get_metrics(y_test, y_pred, "Logistic Regression")

# # F1= 0.919511
# # F1= 0.931996	
# # f1_score(y_test, y_pred)
# result = pl.DataFrame([metrics])
# result

In [ ]:
# fig: Figure = plot_confusion_matrix(result)
# fig.show()